In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import numpy as np
import pandas as pd
import collections
import shutil
import math
import os
import random
import re
import csv
from tensorflow.python.platform import gfile

In [3]:
tf.__version__

'1.4.0'

In [12]:
WORD_VOCAB_FILE = None#'gs://{}/training/data/vocab'.format(BUCKET)
PADWORD = 'ZYXW'
BATCH_SIZE = 1024
FILEPATH = None#'gs://{}/training/data/en_reviews_nlp_with_scores_doclevel.csv'.format(BUCKET)
mdl = 20
#N_WORDS = save_vocab('gs://{}/training/data/{}'.format(BUCKET, FILENAME), 'Tokens', WORD_VOCAB_FILE)

In [17]:
#load dataset from a tab separated file
filename = "sentence_training.csv"
df = pd.read_csv(filename, sep='\t')
df = df.dropna()
#Load categories and corresponding ids from a separate xl file
#label_df= pd.read_excel("BG Category.xlsx").fillna('')

In [22]:
#Making dictionary of key value pair of categories and their ids
#labels1 = label_df['BGSubCategory'].apply(lambda x: x.strip()) 
#ids = label_df['BGSubCategoryId'] 
#zipped = list(zip(ids, labels1))
#labels_dict = dict(zipped)
#print(labels_dict.values())

In [18]:
#Splitting to get all the labels which were generated by API in a single field
labels=df['Labels'].apply(lambda x: x.split(','))
#type(labels)
data = df['Text']
#labels = df['Labels']
labels[5]

['Rooms/Room Size', 'Rooms/Room Lighting', 'Rooms/Room Location']

In [14]:
#Maxm document length without restrictions
#max_document_length = max([len(x.split(" ")) for x in data])
#print(max_document_length)

119


In [19]:
#Multihot Encoding of all labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)


In [13]:
MAX_DOCUMENT_LENGTH = 20
def save_vocab(trainfile, outfilename):
    """Reading file and creating vocab file"""
    if trainfile.startswith('gs://'):
        import subprocess
        tmpfile = "vocab.csv"
        subprocess.check_call("gsutil cp {} {}".format(trainfile, tmpfile).split(" "))
        filename = tmpfile
    else:
        filename = trainfile
    import pandas as pd
    df = pd.read_csv(filename, sep='\t')
    df = df.dropna()
    # the text to be classified
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
    MAX_DOCUMENT_LENGTH) 
    
    vocab_processor.fit(df.iloc[:,0])
    

    with gfile.Open(outfilename, 'wb') as f:
        f.write("{}\n".format(PADWORD))
        for word, index in vocab_processor.vocabulary_._mapping.iteritems():
          f.write("{}\n".format(word))
    nwords = len(vocab_processor.vocabulary_)
    print('{} words into {}'.format(nwords, outfilename))
    return nwords + 2  # PADWORD and <UNK>


In [15]:
vocab_size = save_vocab('sentence_training.csv', 'sentence_training_vocab.csv')

657 words into sentence_training_vocab.csv


In [18]:
#Shuffling of all data so that we can feed unbiased data
#np.random.seed(22)
#shuffle_indices = np.random.permutation(np.arange(len(x_data)))
#x_shuffled = x_data[shuffle_indices]
#y_shuffled = y_output[shuffle_indices]
#print(y_shuffled)

In [19]:
filename = "sentence_training.csv"
dataset = tf.data.TextLineDataset(filename)\
                .skip(1)
#type(dataset)

tensorflow.python.data.ops.dataset_ops.SkipDataset

In [51]:
mlb.transform([df['Labels'][0].split(',')])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [193]:
table = tf.contrib.lookup.index_table_from_file(vocabulary_file='sentence_training_vocab.csv', num_oov_buckets=1, default_value=-1)   
def convert_to_indices(raw_text):
    
    words = tf.string_split([raw_text])
    densewords = tf.sparse_tensor_to_dense(words, default_value=PADWORD)
    numbers = table.lookup(densewords)
    padding = tf.constant([[0,0],[0,MAX_DOCUMENT_LENGTH]])
    padded = tf.pad(numbers, padding)
    sliced = tf.slice(padded, [0,0], [-1, MAX_DOCUMENT_LENGTH])
    print (sliced)
    return sliced

In [198]:

def train_input_fn():
    filename = "sentence_training.csv"
    dataset = tf.data.TextLineDataset(filename)\
                .skip(1)


    # Use `tf.parse_single_example()` to extract data from a `tf.Example`
    # protocol buffer, and perform any additional per-record preprocessing.
    def _parse_data(line):
        """Preprocessing of text on-the-fly"""
        line_split = tf.string_split([line], '\t')
        #print line_split.values
        raw_text = tf.py_func(
            lambda x: ' '.join([i for i in x.split() if len(i)>2]), [line_split.values[0]], tf.string)

        raw_text=convert_to_indices(raw_text)
        # string operations
        #text = tf.squeeze([raw_text], [1])

        labels = tf.py_func(
            lambda x: mlb.transform([x.split(',')]), [line_split.values[1]], tf.int64)

        #score = tf.string_to_number(line_split.values[1])
        #labels = line_split.values[1]
        #return features as dict and score
        return {"raw_text": raw_text}, labels



    # Use `Dataset.map()` to build a pair of a feature dictionary and a label
    # tensor for each example.
    dataset = dataset.map(_parse_data)
    #dataset = dataset.shuffle(buffer_size=1000000)
    dataset = dataset.batch(BATCH_SIZE)
    #dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()


    # `features` is a dictionary in which each value is a batch of values for
    # that feature; `labels` is a batch of labels.
    features, labels = iterator.get_next()
    return features, labels

In [31]:
len(mlb.classes_)

65

In [174]:
column = tf.feature_column.categorical_column_with_identity('raw_text', vocab_size)
    

In [185]:
#tf.logging.set_verbosity(tf.logging.INFO)
#shutil.rmtree("ximi_trained", ignore_errors=True)# start fresh each time
estimator = tf.contrib.estimator.DNNEstimator(  
    head=tf.contrib.estimator.multi_label_head(n_classes=len(mlb.classes_)),
    feature_columns=[column],
    hidden_units=[1024, 512, 256],
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa9d0266dd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpimtG2i', '_save_summary_steps': 100}


In [187]:
train_input = lambda: train_input_fn()
eval_input = lambda: train_input_fn()

In [188]:
def json_serving_input_fn():
    """Build the serving inputs."""
    inputs = {'raw_text': tf.placeholder(tf.string)}

    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

In [189]:
train_spec = tf.estimator.TrainSpec(train_input,
                                  max_steps=1000
                                  )
exporter = tf.estimator.FinalExporter('census',
      json_serving_input_fn)
eval_spec = tf.estimator.EvalSpec(eval_input,
                                steps=10,
                                exporters=[exporter],
                                name='census-eval'
                                )

In [199]:
sess.run(tf.tables_initializer())


In [200]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
Tensor("Slice:0", shape=(?, 20), dtype=int64)


ValueError: Cannot capture a stateful node (name:string_to_index_11/hash_table, type:HashTableV2) by value.

In [194]:
tf.tables_initializer()

<tf.Operation 'init_all_tables_22' type=NoOp>

In [30]:
def input_fn_train: # returns x, y
  pass
estimator.train(input_fn=input_fn_train, steps=100)

def input_fn_eval: # returns x, y
  pass
metrics = estimator.evaluate(input_fn=input_fn_eval, steps=10)
def input_fn_predict: # returns x, None
  pass
predictions = estimator.predict(input_fn=input_fn_predict)

SyntaxError: invalid syntax (<ipython-input-30-9e2995afe265>, line 1)

In [95]:
# `features` is a dictionary in which each value is a batch of values for
# that feature; `labels` is a batch of labels.
for i in range(30):
    features, labels = iterator.get_next()
    print (sess.run([features, labels]))

[{'raw_text': array([[229, 484, 177, 272, 389, 269,  22,  64, 272, 407,  80, 493, 562,
        464, 484,  76, 269, 122, 480, 254]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])]
[{'raw_text': array([[ 89, 484, 256, 411, 658, 230,  74, 369,  22, 521, 595, 112,  49,
        455, 584, 658,   0,   0,   0,   0]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])]
[{'raw_text': array([[118, 466, 584,  22, 514, 595, 149, 182, 150, 144, 584, 523, 434,
        644, 484, 256, 380, 484, 251, 272]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

[{'raw_text': array([[658, 580, 260, 250,  17, 539, 499, 658, 658,  88, 215, 604, 323,
        484,  90, 637,  70, 304, 584, 631]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])]
[{'raw_text': array([[272, 254, 608, 369, 475, 288, 541, 103, 484,  15, 178, 369, 340,
        465, 539,  84, 658,  25, 656, 650]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])]
[{'raw_text': array([[ 74, 450, 459, 157,  96, 594, 291, 595, 484, 591, 658,   0,   0,
          0,   0,   0,   0,   0,   0,   0]])}, array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [29]:
estimator.fit(input_fn = train_input_fn)

ValueError: Items of feature_columns must be a _FeatureColumn. Given (type <class 'numpy.ndarray'>): [ 1  2  3  4  5  6  7  8  9  6 10 11 12  6 13 14 15 16 17  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0].

In [ ]:
# Export the model for serving
feature_spec = {'f_features': x_data}

serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir_base='/tmp/ximi_model' + '/export', 
                            serving_input_receiver_fn=serving_fn)